In [ ]:
import numpy as np
from scipy.stats.mstats import gmean
from scipy.stats import norm

In [ ]:
def GeoAsianCallBSMPricer(strike, expiry, spot, rate, volatility, dividend, steps):
    u = rate - dividend + .5*volatility**2
    a = steps*(steps+1)*(2*steps+1)/6
    v = np.exp(-rate*expiry)*spot*np.exp((steps+1)*u/2 + (a*expiry*volatility**2)/(2*steps**2))
    avg_vol = volatility*np.exp((2*steps+1)/(6*(steps+1)))
    d1 = (1/avg_vol*np.exp(expiry))*(np.log(v/strike) + (rate-dividend+.5*avg_vol**2)*expiry)
    d2 = d1 - avg_vol*np.exp(expiry)
    price = np.exp(-dividend*expiry)*v*norm.cdf(d1) - np.exp(-rate*expiry)*strike*norm.cdf(d2)
    return price

In [ ]:
def call_payoff(spot, strike):
    payoff = max(spot - strike, 0)
    return payoff

def put_payoff(spot, strike):
    payoff = max(strike - spot, 0)
    return payoff

In [ ]:
def PathwiseCVMonteCarloPricer(strike, expiry, spot, rate, vol, div, replications, steps):
    dt = expiry / steps
    disc = np.exp(-rate * dt)
    spotPath = np.zeros((replications, steps))
    spotPath[:,0] = spot
    for j in range(replications):
        arithmetic_prices = np.zeros((replications))
        geo_prices = np.zeros((replications))
        for t in range(1, int(steps)):
            z = np.random.normal(size=int(steps))
            spotPath[j,t]= spotPath[j,t-1] * np.exp((rate - div - 0.5 * vol * vol) * dt + vol * np.sqrt(dt) * z[t])
        arithmetic_prices[j] = call_payoff(np.average(spotPath[j]), strike)
        geo_prices[j] = call_payoff(gmean(spotPath[j]), strike)
        
    GBSM_price = GeoAsianCallBSMPricer(strike, expiry, spot, rate, vol, div, steps)
    price = np.average(arithmetic_prices) + GBSM_price - np.average(geo_prices)

    return price

In [ ]:
def ControlVariatePricer(strike, expiry, spot, rate, volatility, dividend, replications, steps):
    dt = expiry / steps
    nudt = (rate - dividend - 0.5 * volatility * volatility) * dt
    sigsdt = volatility * np.sqrt(dt)
    erddt = np.exp((rate - dividend) * dt)    
    beta = 1.0
    cash_flow_t = np.zeros((replications, ))
    price = 0.0

    for j in range(replications):
        spot_t = spot
        convar = 0.0
        z = np.random.normal(size=int(steps))

        for i in range(int(steps)):
            t = i * dt
            delta = GeoAsianCallBSMPricer(strike, expiry, spot, rate, volatility, dividend, steps)
            spot_tn = spot_t * np.exp(nudt + sigsdt * z[i])
            convar = convar + delta * (spot_tn - spot_t * erddt)
            spot_t = spot_tn

        cash_flow_t[j] = call_payoff(spot_t, strike) + beta * convar

    price = np.exp(-rate * expiry) * cash_flow_t.mean()
    stderr = cash_flow_t.std() / np.sqrt(replications)
    return price, stderr

In [ ]:
strike = 100
spot = 100
vol = .2
rate = .06
div = .03
steps = 10
replications = 10000
expiry = 1

price = PathwiseCVMonteCarloPricer(strike, expiry, spot, rate, vol, div, replications, steps)
print(price)

35.98514125861664
